In [6]:
import pickle as pkl
import os
import sys
import joblib
import numpy as np

In [7]:
def load_pickle_file(file_path):
    """
    Loads a pickle file from the given file path into an object.

    :param file_path: Path to the pickle file
    :return: The object loaded from the pickle file
    """
    try:
        with open(file_path, 'rb') as file:
            obj = pkl.load(file)
        return obj
    except (FileNotFoundError, pkl.PickleError) as e:
        print(f"An error occurred: {e}")
        return None

def load_joblib_pickle(file_path):
    """
    Loads a pickle file using joblib from the given file path.

    :param file_path: Path to the pickle file
    :return: The object loaded from the pickle file
    """
    try:
        obj = joblib.load(file_path)
        return obj
    except (FileNotFoundError, joblib.externals.loky.process_executor.TerminatedWorkerError, Exception) as e:
        print(f"An error occurred: {e}")
        return None

In [8]:
from PIL import Image, ImageDraw
import numpy as np

def annotate_circles(image_path, points, output_path, radius=2, color="red", thickness=2):
    # Open the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Draw circles at each point
    for (x, y) in points:
        top_left = (x - radius, y - radius)
        bottom_right = (x + radius, y + radius)
        for i in range(thickness):  # Draw concentric circles for thickness
            draw.ellipse([top_left[0] - i, top_left[1] - i, bottom_right[0] + i, bottom_right[1] + i], outline=color)

    # Save the annotated image
    image.save(output_path)
    print(f"Annotated image saved to: {output_path}")

In [9]:
file_annots = load_joblib_pickle("../HCM_Dataset/HCM_annots/00003.pkl")[0]
gt_joints = file_annots["gt_joints"].numpy()
for key, _ in file_annots.items():
    print(key)

print(file_annots["scale"])


global_orient
pose
betas
global_trans
cam_eye
scale
img_fname
view_matrix
cam_look_at
cam_up
cam_int
gender
gt_joints
joints_2d
verts
joints_3d
tensor([894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518,
        894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518, 894.9518

In [10]:
print(file_annots["cam_int"][0])
print(file_annots["view_matrix"][20])

tensor([[934.0728,   0.0000, 640.0000],
        [  0.0000, 934.0728, 360.0000],
        [  0.0000,   0.0000,   1.0000]])
tensor([[ 2.2686e-01, -2.6402e-01,  9.3746e-01,  0.0000e+00],
        [-1.3698e-08,  9.6256e-01,  2.7108e-01,  0.0000e+00],
        [-9.7393e-01, -6.1497e-02,  2.1836e-01, -0.0000e+00],
        [-7.1088e+03,  2.8119e+03, -1.1035e+04,  1.0000e+00]],
       dtype=torch.float64)


In [11]:
t=0

for joint_i in range(10):
  np.set_printoptions(suppress=True)
  cam_int = (file_annots["cam_int"][t]).numpy()
  view_matrix = (file_annots["view_matrix"][t]).numpy().T
  joint = gt_joints[t][joint_i]
  hom_joint = np.hstack((joint, np.array([1])))
  cam_joint = np.matmul(view_matrix, hom_joint)
  cam_joint = cam_joint / cam_joint[3]
  cam_joint = cam_joint[:3]
  screen_joint = np.matmul(cam_int, cam_joint)
  screen_joint = screen_joint / screen_joint[2]
  print(np.ndarray.round(screen_joint, decimals=0))

[609. 347.   1.]
[608. 333.   1.]
[618. 323.   1.]
[628. 313.   1.]
[637. 302.   1.]
[644. 297.   1.]
[647. 284.   1.]
[647. 297.   1.]
[652. 302.   1.]
[652. 291.   1.]


In [12]:

t=0

cam_int = (file_annots["cam_int"][t]).numpy()
view_matrix = (file_annots["view_matrix"][t]).numpy().T
joints = gt_joints[t]
hom_joints = np.hstack((joints, np.ones((joints.shape[0], 1))))
cam_joints = (view_matrix@hom_joints.T).T
cam_joints = cam_joints / cam_joints[:, -1][:, np.newaxis]
cam_joints = cam_joints[:, :3]
screen_joints = (cam_int @ cam_joints.T).T
screen_joints = screen_joints / screen_joints[:, -1][:, np.newaxis]
screen_joints = screen_joints[:, :2]
fix_joints = np.vstack(( (np.full((screen_joints.shape[0],), 1280)) - screen_joints[:, 0], screen_joints[:, 1]))
print(fix_joints.T)

[[671.43626062 346.70230515]
 [672.15212717 332.80772555]
 [661.87810815 323.35900122]
 [652.49871241 313.00063468]
 [643.37599345 302.38749705]
 [635.97531961 296.923529  ]
 [633.42168808 284.30609037]
 [633.23764875 296.7219827 ]
 [628.12630148 302.42070165]
 [628.19476489 291.2826805 ]
 [627.03721042 291.53951345]
 [627.48023389 290.09224103]
 [626.28726509 290.35667983]
 [627.47428454 298.92509963]
 [627.00945063 298.05699508]
 [628.19476489 291.2826805 ]
 [626.57323735 291.24601117]
 [627.48023389 290.09224103]
 [625.82640901 290.06471835]
 [625.32923448 290.03153857]
 [624.91045112 289.16675847]
 [650.28539028 307.69778507]
 [650.81552421 309.42711342]
 [665.00988537 327.23718298]
 [650.27731106 348.48204797]
 [647.17274588 348.84934097]
 [644.33152585 351.0222173 ]
 [642.30135429 352.99923764]
 [641.53565269 355.05747038]
 [641.87964945 352.58230971]
 [641.23655076 355.08334616]
 [642.47545807 355.88956584]
 [644.17359548 355.313578  ]
 [643.2666202  353.98606406]
 [642.54215675

In [13]:
annotate_circles("../images/00003/00050_frame.png", file_annots["joints_2d"][50], "./test.png")

Annotated image saved to: ./test.png


In [8]:
cam_path = eg_file_annot[0]["cam_eye"].numpy()
pose_points = eg_file_annot[0]["joints_3d"].numpy()[20]
view_mat = eg_file_annot[0]["view_matrix"].numpy()[20]

In [10]:
pose_points_transformed = apply_view_matrix_to_points(view_mat, pose_points)

In [11]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def plot_3d_points(cam_points):
    """
    Plots an array of points in 3D space.

    :param points: NumPy array or list of points with shape (n, 3), where each point is [x, y, z].
    """
    cam_x, cam_y, cam_z = cam_points[:, 0], cam_points[:, 1], cam_points[:, 2]  # Extract x, y, z coordinates

    # Create a 3D scatter plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(cam_x, cam_y, cam_z, c='red', marker='o', s=50)  # Red circles as markers

    # Set labels for the axes
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # Show the plot
    plt.show()

In [13]:
# for i in range(len(joints)):
#    annotate_circles(f"../images/00005/" + str(i).zfill(5) + "_frame.png", joints[i], f"test_{str(i).zfill(5)}.png")
    

Annotated image saved to: test_00000.png
Annotated image saved to: test_00001.png
Annotated image saved to: test_00002.png
Annotated image saved to: test_00003.png


KeyboardInterrupt: 